In [ ]:
from datetime import date
import pandas as pd
import numpy as np
import yaml
from sqlalchemy import create_engine

In [ ]:
# Cargar configuración
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

In [ ]:

# Crear conexiones
url_fuente = f"postgresql://{config_fuente['user']}:{config_fuente['password']}@{config_fuente['host']}:{config_fuente['port']}/{config_fuente['dbname']}"
url_bodega = f"postgresql://{config_bodega['user']}:{config_bodega['password']}@{config_bodega['host']}:{config_bodega['port']}/{config_bodega['dbname']}"

fuente_conn = create_engine(url_fuente)
bodega_conn = create_engine(url_bodega)

In [ ]:
# Consulta de ejemplo para validar los datos cargados
consulta_validacion = """
SELECT 
    h.hora_total,
    f.fecha,
    ho.hora,
    c.nombre as cliente,
    m.nombre as mensajero,
    s.nombre as sede
FROM hecho_entrega_servicio_hora h
JOIN dim_fecha f ON h.key_dim_fecha = f.key_dim_fecha
JOIN dim_hora ho ON h.key_dim_hora = ho.key_dim_hora
JOIN dim_cliente c ON h.key_dim_cliente = c.key_dim_cliente
JOIN dim_mensajero m ON h.key_dim_mensajero = m.key_dim_mensajero
JOIN dim_sede s ON h.key_dim_sede = s.key_dim_sede
LIMIT 5;
"""

In [ ]:
# Leer datos y dimensiones
df = pd.read_sql(query, fuente_conn)
dim_fecha = pd.read_sql_table('dim_fecha', bodega_conn)
dim_cliente = pd.read_sql_table('dim_cliente', bodega_conn)
dim_mensajero = pd.read_sql_table('dim_mensajero', bodega_conn)
dim_sede = pd.read_sql_table('dim_sede', bodega_conn)
dim_hora = pd.read_sql_table('dim_hora', bodega_conn)

In [ ]:
# Limpiar formato de hora
def limpiar_hora(hora_str):
    try:
        if '.' in str(hora_str):
            return str(hora_str).split('.')[0]
        return str(hora_str)
    except:
        return None

df['hora_estado'] = df['hora_estado'].apply(limpiar_hora)


In [ ]:
# Extraer componentes de hora
def extraer_hora(hora_str):
    try:
        if pd.isna(hora_str):
            return None
        return int(hora_str.split(':')[0])
    except:
        return None

def extraer_minuto(hora_str):
    try:
        if pd.isna(hora_str):
            return None
        return int(hora_str.split(':')[1])
    except:
        return None

def extraer_segundo(hora_str):
    try:
        if pd.isna(hora_str):
            return None
        return int(hora_str.split(':')[2])
    except:
        return None

df['hora'] = df['hora_estado'].apply(extraer_hora)
df['minuto'] = df['hora_estado'].apply(extraer_minuto)
df['segundo'] = df['hora_estado'].apply(extraer_segundo)


In [ ]:
# Convertir fecha_estado a datetime
df['fecha_estado'] = pd.to_datetime(df['fecha_estado']).dt.date
dim_fecha['fecha'] = pd.to_datetime(dim_fecha['fecha']).dt.date

In [ ]:
# Realizar los merges con las dimensiones
hecho_hora = df.merge(
    dim_fecha[['key_dim_fecha', 'fecha']], 
    left_on='fecha_estado', 
    right_on='fecha',
    how='left'
)

hecho_hora = hecho_hora.merge(
    dim_cliente[['key_dim_cliente', 'cliente_id']], 
    on='cliente_id',
    how='left'
)

hecho_hora = hecho_hora.merge(
    dim_mensajero[['key_dim_mensajero', 'mensajero_id']], 
    left_on='mensajero_final_id',
    right_on='mensajero_id',
    how='left'
)

hecho_hora = hecho_hora.merge(
    dim_sede[['key_dim_sede', 'sede_id']],
    on='sede_id',
    how='left'
)

hecho_hora = hecho_hora.merge(
    dim_hora[['key_dim_hora', 'hora']],
    on='hora',
    how='left'
)

In [ ]:
# Calcular tiempo total por hora
def calcular_tiempo_total(grupo):
    return len(grupo)  # Cantidad de servicios por hora

hecho_hora_agrupado = hecho_hora.groupby([
    'key_dim_fecha',
    'key_dim_cliente',
    'key_dim_sede',
    'key_dim_mensajero',
    'key_dim_hora'
]).apply(calcular_tiempo_total).reset_index(name='hora_total')

In [ ]:
# Agregar fecha de carga
hecho_hora_agrupado['saved'] = date.today()

In [ ]:
# Verificaciones
print("\nInformación del DataFrame:")
print(hecho_hora_agrupado.info())

print("\nDistribución de servicios por hora:")
print(hecho_hora_agrupado.groupby('key_dim_hora')['hora_total'].mean())

print("\nVerificar valores nulos:")
print(hecho_hora_agrupado.isnull().sum())


In [ ]:
#Guardar en la bodega
hecho_hora_agrupado.to_sql(
    'hecho_entrega_servicio_hora', 
    bodega_conn, 
    if_exists='replace', 
    index=False
)